1. The first query will use a join between the following two tables: 
hdv_modeled.Results_Beam_DF and ers_modeled.Jurisdiction_Demography. The join
will be across the fipscode fields. We will compare the voting results
for 2016 Presidential Election with given unemployment rate data for each jurisdiction

In [76]:
%%bigquery
CREATE VIEW reporting.Presidential_Election_vs_Unemployment_Rates AS
SELECT DISTINCT t1.state, t1.jname, t1.fipscode, t1.cname, t1.party, t1.year, t1.vote_pct ,t2.unemployment_rate_16, t1.winner
FROM alert-result-266803.hdv_modeled.Results_Beam_DF t1
JOIN alert-result-266803.ers_modeled.Jurisdiction_Demography_SQL_Final t2
ON t1.fipscode = t2.fipscode
WHERE t1.winner = 1 and t1.office = 'President'
ORDER BY t1.state, t1.jname

""


2. This second query will compare results for the House 2016 and 2018 elections with population data from corresponding years. Here, we only look to jurisdictions with a growth rate of at least 5 percent from 2016 to 2018.

In [77]:
%%bigquery
CREATE VIEW reporting.House_Elections_vs_Population_Counts AS
SELECT t1.state, t1.jname, t1.fipscode, t1.cname, t1.party, t1.year, t1.vote_pct ,t2.pop_16 as population, t1.winner
FROM alert-result-266803.hdv_modeled.Results_Beam_DF t1
JOIN alert-result-266803.ers_modeled.Jurisdiction_Demography_SQL_Final t2
ON t1.fipscode = t2.fipscode
WHERE  t1.office = 'US House' and ((t2.pop_18 - t2.pop_16)/(t2.pop_16) > 0.05) and t1.year = 2016


UNION DISTINCT
SELECT t1.state, t1.jname, t1.fipscode, t1.cname, t1.party, t1.year, t1.vote_pct ,t2.pop_18 as population, t1.winner
FROM alert-result-266803.hdv_modeled.Results_Beam_DF t1
JOIN alert-result-266803.ers_modeled.Jurisdiction_Demography_SQL_Final t2
ON t1.fipscode = t2.fipscode
WHERE  t1.office = 'US House' and ((t2.pop_18 - t2.pop_16)/(t2.pop_16) > 0.05) and t1.year = 2018

ORDER BY state, jname

""


3. In this last query, we investigate the average poverty level for both Democratic and Republican wins in the 2018 House Election. The top entry represents the Democratic Count and the bottom entry represents the Republican Count.

In [79]:
%%bigquery
CREATE VIEW reporting.Party_Wins_vs_Poverty_Count AS
SELECT AVG(t2.pov_count_18) as Poverty_AVG
FROM alert-result-266803.hdv_modeled.Results_Beam_DF t1
JOIN alert-result-266803.ers_modeled.Jurisdiction_Demography_SQL_Final t2
ON t1.fipscode = t2.fipscode
WHERE t1.winner = 1 and t1.office = 'US House'  and t1.party = 'Republican' and t1.year = 2018 

UNION DISTINCT
SELECT AVG(t2.pov_count_18) 
FROM alert-result-266803.hdv_modeled.Results_Beam_DF t1
JOIN alert-result-266803.ers_modeled.Jurisdiction_Demography_SQL_Final t2
ON t1.fipscode = t2.fipscode
WHERE t1.winner = 1 and t1.office = 'US House'  and t1.party = 'Democrat' and t1.year = 2018
GROUP BY party

""
